# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,484.296988,0.171232,1234,484.468220,484.125756,968.936439,968.251511
6,914.616850,0.839749,1234,915.456599,913.777101,1830.913197,1827.554202
7,982.813289,0.958814,1234,983.772103,981.854476,1967.544206,1963.708951
8,495.840382,-0.833965,1234,495.006417,496.674347,990.012833,993.348694
9,564.290822,0.480787,1234,564.771609,563.810035,1129.543218,1127.620069
10,899.766937,-0.103278,1234,899.663658,899.870215,1799.327317,1799.740430
11,30.265094,-0.562469,1234,29.702625,30.827563,59.405249,61.655126
12,405.486497,0.321100,1234,405.807598,405.165397,811.615195,810.330794
13,78.859294,-0.568382,1234,78.290912,79.427675,156.581824,158.855351
14,947.423298,-0.798972,1234,946.624327,948.222270,1893.248653,1896.444541


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-484.468220,484.125756
6,-915.456599,913.777101
7,-983.772103,981.854476
8,-495.006417,496.674347
9,-564.771609,563.810035
10,-899.663658,899.870215
11,-29.702625,30.827563
12,-405.807598,405.165397
13,-78.290912,79.427675
14,-946.624327,948.222270


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-484.468220,484.125756
6,-915.456599,913.777101
7,-983.772103,981.854476
8,-495.006417,496.674347
9,-564.771609,563.810035
10,-899.663658,899.870215
11,-29.702625,30.827563
12,-405.807598,405.165397
13,-78.290912,79.427675
14,-946.624327,948.222270


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-484.46821961,  484.12575574],
       [-915.4565986 ,  913.77710089],
       [-983.77210278,  981.85447553],
       [-495.00641665,  496.674347  ],
       [-564.77160898,  563.81003472],
       [-899.66365833,  899.87021478],
       [ -29.70262454,   30.82756275],
       [-405.80759765,  405.16539693],
       [ -78.29091197,   79.4276753 ],
       [-946.62432668,  948.22227029]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,966.936439,-968.251511
6,1828.913197,-1827.554202
7,1965.544206,-1963.708951
8,988.012833,-993.348694
9,1127.543218,-1127.620069
10,1797.327317,-1799.740430
11,57.405249,-61.655126
12,809.615195,-810.330794
13,154.581824,-158.855351
14,1891.248653,-1896.444541


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [16]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [17]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [18]:
data['a']  = np.nan

In [19]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [20]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [21]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
